# Credit Risk Rating Report
This Jupyter Notebook performs a credit risk analysis for a company using financial data provided by the user. The analysis includes calculations of key financial ratios, a Discounted Cash Flow (DCF) valuation, credit risk assessment, and a detailed report with visualizations.

## Steps:
1. Input financial data.
2. Validate and process the data.
3. Perform financial ratio analysis.
4. Perform DCF valuation.
5. Assess credit risk.
6. Generate a report with a detailed analysis and visualizations.

In [1]:
# Import necessary libraries
import logging

import pandas as pd

# Initialize logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Create a simple error handler
class ErrorHandler:
    @staticmethod
    def handle_error(code, message):
        logger.error(f'Error {code}: {message}')
        raise Exception(message)


## 1. Input Financial Data
Please provide the financial data for your company. The data required includes values like revenue, cost of sales, operating expenses, total assets, liabilities, debt, and interest expenses. You can enter the values in the form of a dictionary.

In [2]:
# Collect user input for financial data
def get_financial_data():
    # Simulate user input
    financial_data = {
        'Year': [int(input('Enter first year (e.g., 2024): ')), int(input('Enter second year (e.g., 2023): '))],
        'Revenue': [float(input('Enter revenue for year 1: ')), float(input('Enter revenue for year 2: '))],
        'Cost of Sales': [float(input('Enter cost of sales for year 1: ')), float(input('Enter cost of sales for year 2: '))],
        'Operating Expenses': [float(input('Enter operating expenses for year 1: ')), float(input('Enter operating expenses for year 2: '))],
        'Total Assets': [float(input('Enter total assets for year 1: ')), float(input('Enter total assets for year 2: '))],
        'Total Liabilities': [float(input('Enter total liabilities for year 1: ')), float(input('Enter total liabilities for year 2: '))],
        'Cash': [float(input('Enter cash for year 1: ')), float(input('Enter cash for year 2: '))],
        'Short-term Debt': [float(input('Enter short-term debt for year 1: ')), float(input('Enter short-term debt for year 2: '))],
        'Long-term Debt': [float(input('Enter long-term debt for year 1: ')), float(input('Enter long-term debt for year 2: '))],
        'Interest Expense': [float(input('Enter interest expense for year 1: ')), float(input('Enter interest expense for year 2: '))],
    }
    return pd.DataFrame(financial_data)

# Get the financial data from the user
financial_data = get_financial_data()
logger.info('Financial data loaded.')
financial_data

## 2. Validate and Process Data
After collecting the financial data, we need to validate it and ensure that all entries are positive numbers. If any invalid data is encountered, we will handle it gracefully.

In [3]:
# Validate the financial data
def validate_financial_data(financial_data):
    try:
        for col in financial_data.columns:
            if pd.api.types.is_numeric_dtype(financial_data[col]):
                assert (financial_data[col] >= 0).all(), f'{col} contains negative values'
        logger.info('Financial data validated successfully.')
    except Exception as e:
        ErrorHandler.handle_error(101, f'Data validation failed: {e}')

# Validate the user-provided data
validate_financial_data(financial_data)

## 3. Perform Financial Ratio Analysis
Next, we will calculate key financial ratios including profit margin, debt-to-equity ratio, and interest coverage ratio.

In [4]:
# Performing financial analysis
financial_metrics = pd.DataFrame({
    'Year': financial_data['Year'],
    'Profit Margin': (financial_data['Revenue'] - financial_data['Cost of Sales'] - financial_data['Operating Expenses']) / financial_data['Revenue'],
    'Debt-to-Equity': financial_data['Long-term Debt'] / (financial_data['Total Assets'] - financial_data['Total Liabilities']),
    'Current Ratio': financial_data['Cash'] / financial_data['Short-term Debt'],
    'Interest Coverage': (financial_data['Revenue'] - financial_data['Cost of Sales'] - financial_data['Operating Expenses']) / financial_data['Interest Expense'],
    'Leverage Ratio': financial_data['Total Assets'] / (financial_data['Total Assets'] - financial_data['Total Liabilities'])
})
logger.info('Financial metrics calculated.')
financial_metrics

## 4. Discounted Cash Flow (DCF) Valuation
We will now calculate the Discounted Cash Flow (DCF) for the company as a proxy for its valuation. This is a simplified version for demonstration purposes.

In [5]:
# Simple DCF calculation (for demonstration purposes)
discount_rate = 0.1  # 10% discount rate
cash_flows = [15000, 16000, 17000, 18000, 19000]  # Sample future cash flows for 5 years

# Calculate DCF
dcf_value = sum([cf / (1 + discount_rate) ** (i+1) for i, cf in enumerate(cash_flows)])
logger.info(f'DCF valuation calculated: {dcf_value:.2f}')
dcf_value

## 5. Credit Risk Assessment
We will now assess the credit risk based on key financial metrics like the Debt-to-Equity ratio. The rating will align with the S&P corporate credit rating scale:
- AAA to BBB (Investment Grade)
- BB to B (High Yield / Speculative)
- CCC to C (Distressed)
- D (Default)


In [6]:
# Credit Risk Assessment using S&P Rating Scale
def assess_credit_risk(debt_to_equity_ratio, interest_coverage):
    if debt_to_equity_ratio < 1 and interest_coverage > 5:
        return 'AAA'
    elif debt_to_equity_ratio < 1.5 and interest_coverage > 3:
        return 'AA'
    elif debt_to_equity_ratio < 2 and interest_coverage > 2:
        return 'A'
    elif debt_to_equity_ratio < 3 and interest_coverage > 1:
        return 'BBB'
    elif debt_to_equity_ratio < 4 and interest_coverage > 0.5:
        return 'BB'
    elif debt_to_equity_ratio < 5 and interest_coverage > 0.2:
        return 'B'
    elif debt_to_equity_ratio < 6 and interest_coverage > 0.1:
        return 'CCC'
    elif debt_to_equity_ratio >= 6 or interest_coverage <= 0.1:
        return 'C'
    else:
        return 'D'

# Apply credit risk assessment
financial_metrics['Credit Risk Rating'] = financial_metrics.apply(lambda x: assess_credit_risk(x['Debt-to-Equity'], x['Interest Coverage']), axis=1)
logger.info('Credit risk rating assigned for each year.')
financial_metrics[['Year', 'Debt-to-Equity', 'Interest Coverage', 'Credit Risk Rating']]

## 6. Generate Report
Now, we will generate the credit risk report, including key financial highlights, DCF valuation, credit risk assessment, and any visualizations.

In [7]:
# Generating the credit risk report
report = f"""
# Credit Risk Analysis Report

## I. Company Overview
* Company Name: Example Corp
* Analysis Period: {financial_metrics['Year'][0]} - {financial_metrics['Year'][1]}

## II. Financial Highlights
### A. Key Financial Metrics
{financial_metrics.to_markdown(index=False)}

### B. Valuation Summary
* DCF Value: {dcf_value:.2f}

## III. Credit Risk Assessment
* Credit Risk Rating: {financial_metrics['Credit Risk Rating'].iloc[0]}, {financial_metrics['Credit Risk Rating'].iloc[1]}

## IV. Model Summary
* Valuation Model: DCF
* Risk Assessment Model: Debt-to-Equity and Interest Coverage

## V. Visualizations
```python
# Generate financial ratio bar chart
financial_metrics.plot(x='Year', y=['Debt-to-Equity', 'Interest Coverage'], kind='bar', figsize=(10,6))
plt.title('Financial Ratios (Debt-to-Equity and Interest Coverage)')
plt.show()
``` 
"""
logger.info('Report generated.')
print(report)